In [ ]:
import z3
from z3 import Optimize, Int, Real, If, ToReal, ToInt, sat

def _calculate_tax_internal(
    basic_living_exp_per_person: int = 210000,           # 每人基本生活費用
    savings_investment_deduction_limit: int = 270000,      # 儲蓄投資扣除上限
    disability_deduction_per_person: int = 218000,         # 每位身心障礙者扣除額
    education_deduction_per_student: int = 25000,          # 每位學生教育扣除額
    long_term_care_deduction_per_person: int = 120000,     # 每人長期照護扣除額
    rent_deduction_limit: int = 180000,                    # 房屋租金特別扣除上限

    personal_exemption_under70: int = 97000,               # 70歲以下個人免稅額
    personal_exemption_over70: int = 145500,               # 70歲以上個人免稅額

    standard_deduction_single: int = 131000,               # 單身標準扣除額
    standard_deduction_married: int = 262000,              # 已婚標準扣除額
    salary_special_deduction_max: int = 218000,            # 薪資特別扣除額上限

    bracket1_upper: int = 590000,                          # 速算級距1上限
    bracket2_upper: int = 1330000,                         # 速算級距2上限
    bracket3_upper: int = 2660000,                         # 速算級距3上限
    bracket4_upper: int = 4980000,                         # 速算級距4上限

    bracket1_rate: float = 0.05,                           # 速算級距1稅率
    bracket2_rate: float = 0.12,                           # 速算級距2稅率
    bracket3_rate: float = 0.20,                           # 速算級距3稅率
    bracket4_rate: float = 0.30,                           # 速算級距4稅率
    bracket5_rate: float = 0.40,                           # 速算級距5稅率

    bracket1_sub: int = 0,                                 # 速算級距1扣減額
    bracket2_sub: int = 41300,                             # 速算級距2扣減額
    bracket3_sub: int = 147700,                            # 速算級距3扣減額
    bracket4_sub: int = 413700,                            # 速算級距4扣減額
    bracket5_sub: int = 911700,                            # 速算級距5扣減額

    # ===== 使用者申報資料 =====
    is_married: bool = False,                              # 是否已婚
    salary_self: int = 0,                                  # 本人薪資所得
    salary_spouse: int = 0,                                # 配偶薪資所得
    salary_dep: int = 0,                                   # 受扶養親屬薪資所得
    interest_income: int = 0,                              # 利息所得
    stock_dividend: int = 0,                               # 股票股利所得
    house_transaction_gain: int = 0,                       # 房屋交易所得
    other_income: int = 0,                                 # 其他所得

    cnt_under_70: int = 0,                                 # 70歲以下人數
    cnt_over_70: int = 0,                                  # 70歲以上人數

    use_itemized: bool = False,                            # 是否使用列舉扣除
    itemized_deduction: int = 0,                           # 列舉扣除額

    property_loss_deduction: int = 0,                      # 財產損失扣除額
    disability_count: int = 0,                             # 身心障礙者人數
    education_count: int = 0,                              # 教育學費適用人數
    education_fee: int = 0,                                # 實際支付教育費用
    preschool_count: int = 0,                              # 幼兒人數
    long_term_care_count: int = 0,                         # 長期照護適用人數
    rent_deduction: int = 0,                               # 房屋租金支出

    free_vars: list = None                               # 指定哪些變數為自由變數（例如 ["salary_spouse", "interest_income"]）
):
    """
    計算綜合所得稅（內部計算函式）

    除了計算最終應納稅額與淨課稅所得外，
    若 free_vars 不為空，會利用 Optimize 模組與迴圈方式找出使稅額最小的解，
    並回傳所有參數的最終最佳解。其中同時建立一個比較字典，
    僅列出那些「最佳解值」與「原始輸入值」不同的參數，並特別標記 free_vars 裡的變數。

    傳回:
      (final_tax, net_taxable_income, final_params, differences)
        final_tax: 最終應納稅額
        net_taxable_income: 淨課稅所得
        final_params: dict，參數名稱對應 { "value": <最佳解值>, "type": "free" 或 "fixed" }
        differences: dict，只顯示有變更的參數，內容為 { "original": <原始值>, "optimized": <最佳解值>, "difference": <差異> }
    """
    if free_vars is None:
        free_vars = []
        
    # 印出使用者輸入
    user_input = {
        "is_married": is_married,
        "salary_self": salary_self,
        "salary_spouse": salary_spouse,
        "salary_dep": salary_dep,
        "interest_income": interest_income,
        "stock_dividend": stock_dividend,
        "house_transaction_gain": house_transaction_gain,
        "other_income": other_income,
        "cnt_under_70": cnt_under_70,
        "cnt_over_70": cnt_over_70,
        "use_itemized": use_itemized,
        "itemized_deduction": itemized_deduction,
        "property_loss_deduction": property_loss_deduction,
        "disability_count": disability_count,
        "education_count": education_count,
        "education_fee": education_fee,
        "preschool_count": preschool_count,
        "long_term_care_count": long_term_care_count,
        "rent_deduction": rent_deduction,
        "free_vars": free_vars
    }
    print("你的輸入為：")
    print(user_input)
    print("--------------------------------------------------\n")
    
    opt = Optimize()

    # === 1. 定義 Z3 變數 ===
    salary_self_z   = Int('salary_self_z')
    salary_spouse_z = Int('salary_spouse_z')
    salary_dep_z    = Int('salary_dep_z')
    interest_z      = Int('interest_z')
    stock_div_z     = Int('stock_div_z')
    house_gain_z    = Int('house_gain_z')
    other_income_z  = Int('other_income_z')
    itemized_ded_z  = Int('itemized_ded_z')
    prop_loss_ded_z = Int('prop_loss_ded_z')
    rent_ded_z      = Int('rent_ded_z')

    total_income_z  = Int('total_income_z')

    self_salary_special_deduction_z   = Int('self_salary_special_deduction_z')
    spouse_salary_special_deduction_z = Int('spouse_salary_special_deduction_z')
    dep_salary_special_ded_z          = Int('dep_salary_special_ded_z')

    self_salary_after_ded_z   = Int('self_salary_after_ded_z')
    spouse_salary_after_ded_z = Int('spouse_salary_after_ded_z')
    dep_salary_after_ded_z    = Int('dep_salary_after_ded_z')

    total_exemption_z    = Int('total_exemption_z')
    standard_deduction_z = Int('standard_deduction_z')
    chosen_deduction_z   = Int('chosen_deduction_z')

    savings_investment_deduction_z = Int('savings_investment_deduction_z')
    disability_deduction_z         = Int('disability_deduction_z')
    education_deduction_z          = Int('education_deduction_z')

    preschool_count_z     = Int('preschool_count_z')
    preschool_deduction_z = Int('preschool_deduction_z')

    long_term_care_deduction_z = Int('long_term_care_deduction_z')
    rent_deduction_z_lim       = Int('rent_deduction_z_lim')

    total_people_z = Int('total_people_z')
    basic_living_exp_total_z = Int('basic_living_exp_total_z')
    base_deductions_var_z = Int('base_deductions_var_z')
    basic_living_exp_diff_z = Int('basic_living_exp_diff_z')
    total_deduction_z = Int('total_deduction_z')
    net_taxable_income_z = Int('net_taxable_income_z')
    net_taxable_nonneg_z = Int('net_taxable_nonneg_z')

    rate_calc_r = Real('rate_calc_r')
    final_tax_z = Int('final_tax_z')

    # === 2. 建立「輸入值 -> Z3 變數」及範圍檢查 ===
    params = {
        "salary_self": (salary_self_z, salary_self, [lambda var: var >= 0]),
        "salary_spouse": (salary_spouse_z, salary_spouse, [lambda var: var >= 0]),
        "salary_dep": (salary_dep_z, salary_dep, [lambda var: var >= 0]),
        "interest_income": (interest_z, interest_income, [lambda var: var >= 0]),
        "stock_dividend": (stock_div_z, stock_dividend, [lambda var: var >= 0]),
        "house_transaction_gain": (house_gain_z, house_transaction_gain, [lambda var: var >= 0]),
        "other_income": (other_income_z, other_income, [lambda var: var >= 0]),
        "itemized_deduction": (itemized_ded_z, itemized_deduction, [lambda var: var >= 0]),
        "property_loss_deduction": (prop_loss_ded_z, property_loss_deduction, [lambda var: var >= 0]),
        "rent_deduction": (rent_ded_z, rent_deduction, [lambda var: var >= 0, lambda var: var <= 10000000]),
        "cnt_under_70": (Int('cnt_under_70_z'), cnt_under_70, [lambda var: var >= 0]),
        "cnt_over_70": (Int('cnt_over_70_z'), cnt_over_70, [lambda var: var >= 0]),
        "disability_count": (Int('disability_count_z'), disability_count, [lambda var: var >= 0]),
        "education_count": (Int('education_count_z'), education_count, [lambda var: var >= 0]),
        "education_fee": (Int('education_fee_z'), education_fee, [lambda var: var >= 0]),
        "preschool_count": (Int('preschool_count_z'), preschool_count, [lambda var: var >= 0]),
        "long_term_care_count": (Int('long_term_care_count_z'), long_term_care_count, [lambda var: var >= 0])
    }
    
    # 若該參數不在 free_vars 中，直接固定；若在 free_vars 中，僅檢查範圍
    for param_name, (z3_var, value, constraints) in params.items():
        if param_name in free_vars:
            for cons in constraints:
                opt.add(cons(z3_var))
        else:
            opt.add(z3_var == value)
            for cons in constraints:
                opt.add(cons(z3_var))

    # === 其他約束（與原邏輯保持一致） ===
    opt.add(self_salary_special_deduction_z == If(salary_self_z >= salary_special_deduction_max, salary_special_deduction_max, salary_self_z))
    opt.add(spouse_salary_special_deduction_z == If(salary_spouse_z >= salary_special_deduction_max, salary_special_deduction_max, salary_spouse_z))
    opt.add(dep_salary_special_ded_z == If(salary_dep_z >= salary_special_deduction_max, salary_special_deduction_max, salary_dep_z))

    opt.add(self_salary_after_ded_z == salary_self_z - self_salary_special_deduction_z)
    opt.add(spouse_salary_after_ded_z == salary_spouse_z - spouse_salary_special_deduction_z)
    opt.add(dep_salary_after_ded_z == salary_dep_z - dep_salary_special_ded_z)

    opt.add(self_salary_after_ded_z >= 0)
    opt.add(spouse_salary_after_ded_z >= 0)
    opt.add(dep_salary_after_ded_z >= 0)

    opt.add(total_income_z == (self_salary_after_ded_z + spouse_salary_after_ded_z + dep_salary_after_ded_z + interest_z + stock_div_z + house_gain_z + other_income_z))

    opt.add(total_exemption_z == (cnt_under_70 * personal_exemption_under70 + cnt_over_70 * personal_exemption_over70))
    opt.add(standard_deduction_z == (standard_deduction_married if is_married else standard_deduction_single))
    opt.add(chosen_deduction_z == If(use_itemized, itemized_ded_z , standard_deduction_z))

    interest_plus_div_z = Int('interest_plus_div_z')
    opt.add(interest_plus_div_z == (interest_z + stock_div_z))
    opt.add(savings_investment_deduction_z == If(interest_plus_div_z <= savings_investment_deduction_limit, interest_plus_div_z, savings_investment_deduction_limit))
    opt.add(disability_deduction_z == disability_count * disability_deduction_per_person)
    opt.add(education_deduction_z == If(education_fee <= 0, 0, If(education_fee >= education_count * education_deduction_per_student, education_count * education_deduction_per_student, education_fee)))
    opt.add(preschool_count_z == preschool_count)
    opt.add(preschool_deduction_z == If(preschool_count_z <= 0, 0, If(preschool_count_z == 1, 150000, 150000 + (preschool_count_z - 1) * 225000)))
    opt.add(long_term_care_deduction_z == long_term_care_count * long_term_care_deduction_per_person)
    opt.add(rent_deduction_z_lim == If(rent_ded_z >= rent_deduction_limit, rent_deduction_limit, rent_ded_z))
    opt.add(total_people_z == cnt_under_70 + cnt_over_70)
    opt.add(basic_living_exp_total_z == total_people_z * basic_living_exp_per_person)
    opt.add(base_deductions_var_z == (total_exemption_z + chosen_deduction_z + savings_investment_deduction_z + disability_deduction_z + education_deduction_z + preschool_deduction_z + long_term_care_deduction_z + rent_deduction_z_lim))
    opt.add(basic_living_exp_diff_z == If(basic_living_exp_total_z > base_deductions_var_z, basic_living_exp_total_z - base_deductions_var_z, 0))
    opt.add(total_deduction_z == base_deductions_var_z + prop_loss_ded_z + basic_living_exp_diff_z)
    opt.add(net_taxable_income_z == total_income_z - total_deduction_z)
    opt.add(net_taxable_nonneg_z == If(net_taxable_income_z < 0, 0, net_taxable_income_z))
    
    x = ToReal(net_taxable_nonneg_z)
    opt.add(rate_calc_r == If(x <= bracket1_upper,
                              (x * bracket1_rate) - bracket1_sub,
                           If(x <= bracket2_upper,
                              (x * bracket2_rate) - bracket2_sub,
                           If(x <= bracket3_upper,
                              (x * bracket3_rate) - bracket3_sub,
                           If(x <= bracket4_upper,
                              (x * bracket4_rate) - bracket4_sub,
                              (x * bracket5_rate) - bracket5_sub)))))
    safe_tax_r = If(rate_calc_r < 0, 0, rate_calc_r)
    opt.add(final_tax_z == ToInt(safe_tax_r))
    
    # === 印出優化過程 ===
    print("=== 優化過程開始 ===\n")

    # 先利用 push/pop 固定 free_vars 得到「初始稅額 (固定參數)」
    if free_vars:
        opt.push()
        for param in free_vars:
            z3_var, orig_value, _ = params[param]
            opt.add(z3_var == orig_value)
        if opt.check() == sat:
            fixed_model = opt.model()
            fixed_tax = fixed_model[final_tax_z].as_long()
        else:
            fixed_tax = None
        opt.pop()
    else:
        if opt.check() == sat:
            fixed_tax = opt.model()[final_tax_z].as_long()
        else:
            fixed_tax = None
    print("你的稅務運算結果為：{}，此為未放行 free_vars 的初始計算值。".format(fixed_tax))
    print("--------------------------------------------------\n")

    print("選擇可調整變數為：{}".format(free_vars))

    # 進入最佳化迭代，動態更新 iteration
    iteration = 0
    while True:
        if opt.check() != sat:
            print("第 {} 次最佳化: unsat，無解，已是最佳解".format(iteration + 1))
            break
        m = opt.model()
        current_tax = m[final_tax_z].as_long()
        # 第一次或後續找到更低解時印出結果
        if iteration == 0 or current_tax < best_tax:
            best_tax = current_tax  
            print("----- 第 {} 次最佳化 -----".format(iteration + 1))
            print("當前稅額: {}".format(current_tax))
            iter_differences = {}
            for param_name, (z3_var, orig_value, _) in params.items():
                try:
                    opt_value = m.eval(z3_var).as_long()
                except Exception:
                    opt_value = None
                if opt_value is not None and opt_value != orig_value:
                    iter_differences[param_name] = {
                        "original": orig_value,
                        "optimized": opt_value,
                        "difference": opt_value - orig_value
                    }
            if iter_differences:
                print("變更的變數:")
                for key, diff in iter_differences.items():
                    print("  {}: 原始 = {}, 優化後 = {}, 差異 = {}".format(key, diff["original"], diff["optimized"], diff["difference"]))
            print("-------------------------")
        else:
            print("第 {} 次最佳化: 無變化，已是最佳解".format(iteration + 1))
            break
        opt.add(final_tax_z < best_tax)
        iteration += 1


    print("\n=== 最終最佳解，經過 {} 次最佳化，最終稅額 = {} ===".format(iteration, best_tax))
    print("113年度試算  = {}\n".format(best_tax))
    print("最佳化後的結果：{}".format(best_tax))
    print("--------------------------------------------------\n")

    # 建立最終參數字典與差異比較（僅顯示有變更的）
    final_params = {}
    differences = {}
    for param_name, (z3_var, orig_value, _) in params.items():
        try:
            opt_value = m.eval(z3_var).as_long()
        except Exception:
            opt_value = None
        final_params[param_name] = {
            "value": opt_value,
            "type": "free" if param_name in free_vars else "fixed"
        }
        if opt_value is not None and opt_value != orig_value:
            differences[param_name] = {
                "original": orig_value,
                "optimized": opt_value,
                "difference": opt_value - orig_value
            }
    return best_tax, m[net_taxable_income_z].as_long(), final_params, differences

    
def calculate_comprehensive_income_tax(**kwargs):
    """
    計算綜合所得稅（對外API函式）

    傳入參數參考 _calculate_tax_internal 函式之說明。
    若淨所得超過 1,330,000（即進入20%以上級距），則長期照護及房租扣除不適用，
    此時會以調整後參數進行第二次試算。

    若使用者在 kwargs 中指定 free_vars（例如 free_vars=["salary_spouse", "interest_income"]），
    除了傳回最終應納稅額外，亦會傳回最佳解參數與原始值比較結果（僅顯示有變更者）。

    傳回:
      若有 free_vars 則回傳 (最終應納稅額, 參數解 dict, 差異比較 dict)
      否則僅回傳 最終應納稅額
    """
    free_vars = kwargs.get("free_vars", None)
    tax_val, net_income, final_params, differences = _calculate_tax_internal(**kwargs)
    if net_income is not None and net_income >= 1330000:
        kwargs_modified = kwargs.copy()
        kwargs_modified['long_term_care_count'] = 0
        kwargs_modified['rent_deduction'] = 0
        tax_val2, net_income2, final_params2, differences2 = _calculate_tax_internal(**kwargs_modified)
        return (tax_val2, final_params2, differences2) if free_vars else tax_val2
    else:
        return (tax_val, final_params, differences) if free_vars else tax_val


In [29]:
tax = calculate_comprehensive_income_tax(
        is_married=True,
        salary_self=795785,
        salary_spouse=1589914,
        salary_dep=119016, 
        interest_income=35504,
        stock_dividend=33616,
        house_transaction_gain=674194,
        other_income=152873,
        cnt_under_70=1,
        cnt_over_70=3,
        use_itemized=True,
        itemized_deduction=441883,
        property_loss_deduction=181269,
        disability_count=1,
        education_count=0,
        education_fee=0,
        preschool_count=2,
        long_term_care_count=0,
        rent_deduction=88155,
        free_vars = ['salary_self','salary_spouse']
    )

print(tax)

你的輸入為：
{'is_married': True, 'salary_self': 795785, 'salary_spouse': 1589914, 'salary_dep': 119016, 'interest_income': 35504, 'stock_dividend': 33616, 'house_transaction_gain': 674194, 'other_income': 152873, 'cnt_under_70': 1, 'cnt_over_70': 3, 'use_itemized': True, 'itemized_deduction': 441883, 'property_loss_deduction': 181269, 'disability_count': 1, 'education_count': 0, 'education_fee': 0, 'preschool_count': 2, 'long_term_care_count': 0, 'rent_deduction': 88155, 'free_vars': ['salary_self', 'salary_spouse']}
--------------------------------------------------

=== 優化過程開始 ===

你的稅務運算結果為：71375，此為未放行 free_vars 的初始計算值。
--------------------------------------------------

選擇可調整變數為：['salary_self', 'salary_spouse']
----- 第 1 次最佳化 -----
當前稅額: 0
變更的變數:
  salary_self: 原始 = 795785, 優化後 = 1228740, 差異 = 432955
  salary_spouse: 原始 = 1589914, 優化後 = 0, 差異 = -1589914
-------------------------
第 2 次最佳化: unsat，無解，已是最佳解

=== 最終最佳解，經過 1 次最佳化，最終稅額 = 0 ===
113年度試算  = 0

最佳化後的結果：0
--------------------------